# Project Part 2 Solutions

## (a)

In [20]:
import numpy as np

file=np.load("part2.npz")
beta_old=file['beta_old']
N=file['N']
Svc_0_PMF=file['Svc_0_PMF']
Lc=file['Lc']
Ic_0=file['Ic_0']
gamma=file['gamma']
L_observed=file['L_observed']
nb_nodes = Svc_0_PMF.shape[0]

In [2]:
def SIR(t, Z, N, betas, gamma):
    (S_vec, I_vec, R) = (Z[:16].reshape(4, 4), Z[16:20], Z[-1])
    I = I_vec.sum()
    dSdt = -betas*S_vec*I/N 
    dIdt = -dSdt.sum(axis=0)-gamma*I_vec
    dRdt = np.array([gamma*I])
    return np.concatenate((dSdt.flatten(), dIdt, dRdt))

In [128]:
from scipy.integrate import solve_ivp
from scipy.optimize import fmin
from sklearn.metrics import mean_squared_error

Svc_0_PMF /= Svc_0_PMF.sum(axis=(1, 2), keepdims=True)
Svc_0 = (N-Ic_0.sum(axis=1))[:, np.newaxis, np.newaxis]*Svc_0_PMF
Svc_0 = Svc_0.round().astype(int)
R0 = np.zeros((nb_nodes)).astype(int)

t = np.linspace(0, 199, 200)

S = np.zeros((nb_nodes, 4, 4, len(t)))
I = np.zeros((nb_nodes, 4, len(t)))
R = np.zeros((nb_nodes, len(t)))
L = np.zeros((nb_nodes, len(t)))

for N_node, node_index in zip(N, range(nb_nodes)):

    initial_conditions = np.concatenate(
        (Svc_0[node_index].flatten(), Ic_0[node_index], [R0[node_index]]))

    soln = solve_ivp(
        SIR, (t[0], t[9]), initial_conditions, args=(N_node, beta_old, gamma), t_eval=t[:10])

    S[node_index, :, :, :10] = soln.y[:16].reshape(4, 4, -1)
    I[node_index, :, :10] = soln.y[16:20]
    R[node_index, :10] = soln.y[20]
    L[node_index, :10] = Lc@soln.y[16:20]

alpha_predicted = np.zeros((nb_nodes))
def loss_func(alpha):
    
    ###### WRITE YOUR FUNCTION HERE

    soln_obj=solve_ivp(SIR, (t[9], t[29]), np.concatenate((S[node_index, :, :, 9:10].flatten(),\
                                                           I[node_index, :, 9:10].flatten(),R[node_index, 9:10]))\
                       , args=(N_node, beta_old*alpha, gamma), t_eval=t[9:29])
    '''
    check the value below I'm not sure how to calculate it
    ''' 
    
    Lpred = Lc@soln_obj.y[16:20]

    mse = mean_squared_error(L_observed[node_index][9:29].flatten(), (Lpred).flatten()) 
 
  

    return mse


for N_node, node_index in zip(N, range(nb_nodes)):
    alpha_predicted[node_index] = fmin(loss_func, 0.5, disp=False)


In [129]:
alpha_predicted

array([0.23352203, 0.3860054 , 0.22739868, 0.70975647, 0.67788391,
       0.20684509, 0.35756836, 0.35657654, 0.37946777, 0.18708496,
       0.28283691, 0.32966309, 0.31420898, 0.2162323 , 0.13437347,
       0.24160156, 0.20031738, 0.55296021, 0.58077087, 0.18221436,
       0.26948242, 0.30437622, 0.48616943, 0.30966797, 0.33355408,
       0.16851807, 0.36960449, 0.21408691, 0.47091064, 0.66050568,
       0.29938965, 0.30321045, 0.38393707, 0.17612305, 0.59567566,
       0.63848572, 0.39033203, 0.17086182, 0.28800049, 0.86357422,
       0.41445923, 0.43756104, 0.42476196, 0.18603516, 0.24894104,
       0.43441162, 0.69438477, 0.32382202, 0.24464111, 0.21859131,
       0.14959717, 0.2128418 , 0.32988281, 0.26757812, 0.17961426,
       0.65092468, 0.19025879, 0.52108459, 0.27169952, 0.55423279,
       0.36207275, 0.28662109, 0.3498291 , 0.41787415, 0.25991821,
       0.50830688, 0.20330505, 0.41876221, 0.45647888, 0.16158829,
       0.16303711, 0.33074341, 0.2930542 , 0.44996338, 0.40922

## (b)

In [ ]:
def g(money):
    alpha = 1/np.log2(2*10**(-3)*money+2)
    return alpha

def g_inv(alpha):
    money = (2**(1/alpha)-2)/(2*10**(-3))
    return money
    import matplotlib.pyplot as plt 

def scatter_plot(alpha_afterNPI):
    plt.figure()
    colors=(beta_old[np.newaxis,:,:]*Svc_0_PMF).mean(axis=(1,2))
    plt.scatter(alpha_predicted, alpha_afterNPI, s=N/500, c=colors, cmap='jet', alpha=0.5)
    plt.xlabel(r"$\alpha_{own}$")
    plt.ylabel(r"$\alpha_{after\ NPI}$")

def total_recovered_difference(alpha_afterNPI):
    
    ###### WRITE YOUR FUNCTION HERE
    
    return difference

### Policy #1

i.

ii.

### Policy #2

i. 

ii.

### Policy #3

i.

ii.

### Policy #4

i.

ii.

## (c)

### Policy 1

### Policy 2

### Policy 3

### Policy 4